## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install roboflow pillow matplotlib seaborn tqdm numpy

In [ ]:
# Clone the git repository with the code
import os
if not os.path.exists('Dice-Detection'):
    !git clone https://github.com/Adr44mo/Dice-Detection.git
    %cd Dice-Detection
else:
    %cd Dice-Detection

# Add src to path
import sys
sys.path.append('./src')

## 2. Import Libraries

In [ ]:
import torch
import torch.utils.data as data
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
import os

# Import custom modules
from src.dataset import DiceDetectionDataset, collate_fn
from src.model import get_fasterrcnn_model, save_model_checkpoint, get_fasterrcnn_mobilenet
from src.training import train_one_epoch, evaluate, get_optimizer, get_lr_scheduler
from src.metrics import evaluate_map, print_metrics
from src.augmentation import ClassAwareSampler, MosaicAugmentation, apply_random_augmentations
from src.visualization import (
    plot_class_distribution,
    plot_training_history,
    display_sample_batch,
    visualize_predictions,
    plot_ap_comparison
)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 3. Augmentation Configuration

Use the flags below to toggle different augmentation techniques.

In [ ]:
# AUGMENTATION CONFIGURATION
# -------------------------
# Set these flags to enable/disable specific techniques

# 1. Class-Aware Sampling
# Balances training by oversampling minority classes
USE_CLASS_AWARE_SAMPLING = True

# 2. Mosaic Augmentation
# Combines 4 images into a 2x2 grid
USE_MOSAIC_AUGMENTATION = True
MOSAIC_PROB = 0.5

# 3. Random Augmentations
# Applies random flips, color jitter, etc.
USE_RANDOM_AUGMENTATION = True

print("Training Configuration:")
print(f"  Class-Aware Sampling: {'ENABLED' if USE_CLASS_AWARE_SAMPLING else 'DISABLED'}")
print(f"  Mosaic Augmentation:  {'ENABLED' if USE_MOSAIC_AUGMENTATION else 'DISABLED'}")
print(f"  Random Augmentations: {'ENABLED' if USE_RANDOM_AUGMENTATION else 'DISABLED'}")

## 4. Download Dataset from Roboflow

In [ ]:
from roboflow import Roboflow

# TODO: Add your Roboflow API key
rf = Roboflow(api_key="YOUR_API_KEY")

# Download the dice dataset
project = rf.workspace("workspace-spezm").project("dice-0sexk")
dataset = project.version(2).download("coco")

print(f"Dataset downloaded to: {dataset.location}")

## 5. Prepare Datasets

In [ ]:
# Set dataset paths
DATASET_PATH = dataset.location
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
VAL_PATH = os.path.join(DATASET_PATH, "valid")
TEST_PATH = os.path.join(DATASET_PATH, "test")

# Check if test set exists
has_test_set = os.path.exists(TEST_PATH) and os.path.exists(os.path.join(TEST_PATH, "_annotations.coco.json"))
print(f"Test set available: {has_test_set}")

# Create datasets
train_dataset = DiceDetectionDataset(
    root_dir=TRAIN_PATH,
    annotation_file="_annotations.coco.json",
    split="train"
)

val_dataset = DiceDetectionDataset(
    root_dir=VAL_PATH,
    annotation_file="_annotations.coco.json",
    split="val"
)

# Create test dataset if available
if has_test_set:
    test_dataset = DiceDetectionDataset(
        root_dir=TEST_PATH,
        annotation_file="_annotations.coco.json",
        split="test"
    )
    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")
else:
    test_dataset = None
    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    print(f"No test set - will use validation set for final evaluation")

print(f"Number of classes: {train_dataset.num_classes}")

## 6. Flexible Augmentation Wrapper

In [ ]:
class AugmentedDataset(torch.utils.data.Dataset):
    """Wrapper dataset that applies configured augmentations"""
    
    def __init__(self, base_dataset, use_mosaic=True, mosaic_prob=0.5, use_random=True):
        self.base_dataset = base_dataset
        self.use_mosaic = use_mosaic
        self.use_random = use_random
        
        if use_mosaic:
            self.mosaic = MosaicAugmentation(
                base_dataset,
                output_size=(640, 640),
                prob=mosaic_prob
            )
    
    def __len__(self):
        return len(self.base_dataset)
    
    def __getitem__(self, idx):
        # Apply Mosaic if enabled
        if self.use_mosaic:
            image, target = self.mosaic(idx)
        else:
            image, target = self.base_dataset[idx]
        
        # Convert tensor to PIL if needed for augmentations
        if isinstance(image, torch.Tensor):
            from torchvision.transforms import functional as F
            image = F.to_pil_image(image)
        
        # Apply Random Augmentations if enabled
        if self.use_random:
            image, target = apply_random_augmentations(image, target)
        
        # Convert back to tensor
        from torchvision.transforms import ToTensor
        image = ToTensor()(image)
        
        return image, target

# Create augmented training dataset based on config
augmented_train_dataset = AugmentedDataset(
    train_dataset,
    use_mosaic=USE_MOSAIC_AUGMENTATION,
    mosaic_prob=MOSAIC_PROB,
    use_random=USE_RANDOM_AUGMENTATION
)

print(f"Created dataset wrapper with:")
print(f"  Mosaic: {USE_MOSAIC_AUGMENTATION}")
print(f"  Random Augmentations: {USE_RANDOM_AUGMENTATION}")

## 7. Create Data Loaders with Optional Sampling

In [ ]:
# Hyperparameters
BATCH_SIZE = 4
NUM_WORKERS = 2
NUM_EPOCHS = 10
LEARNING_RATE = 0.005
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Configure Sampler
if USE_CLASS_AWARE_SAMPLING:
    train_sampler = ClassAwareSampler(
        train_dataset,
        samples_per_epoch=len(train_dataset) * 2,  # Sample more to see each class
        balance_by='dice_value'
    )
    shuffle = False  # Don't shuffle when using custom sampler
    print(f"Using Class-Aware Sampler ({len(train_sampler)} samples/epoch)")
else:
    train_sampler = None
    shuffle = True
    print("Using Standard Random Sampling")

# Create data loaders
train_loader = data.DataLoader(
    augmented_train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=shuffle,
    sampler=train_sampler,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)

val_loader = data.DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

## 8. Initialize Model

In [ ]:
# Create model
model = get_fasterrcnn_mobilenet(
    num_classes=train_dataset.num_classes,
    pretrained=True,
    trainable_backbone_layers=3
)

model.to(DEVICE)

# Setup optimizer and scheduler
optimizer = get_optimizer(model, lr=LEARNING_RATE)
lr_scheduler = get_lr_scheduler(optimizer, step_size=3, gamma=0.1)

print(f"Model initialized on {DEVICE}")

## 9. Training Loop

In [ ]:
# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'learning_rate': []
}

best_val_loss = float('inf')
CHECKPOINT_DIR = "checkpoints_comparison"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("Starting training...\n")

In [ ]:
for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}")
    print(f"{'='*60}")
    
    # Train
    train_metrics = train_one_epoch(
        model, optimizer, train_loader, DEVICE, epoch + 1
    )
    
    # Evaluate
    val_metrics = evaluate(model, val_loader, DEVICE)
    
    # Update learning rate
    lr_scheduler.step()
    
    # Record history
    history['train_loss'].append(train_metrics['loss'])
    history['val_loss'].append(val_metrics['val_loss'])
    history['learning_rate'].append(optimizer.param_groups[0]['lr'])
    
    # Print summary
    print(f"\nEpoch {epoch + 1} Summary:")
    print(f"  Train Loss: {train_metrics['loss']:.4f}")
    print(f"  Val Loss: {val_metrics['val_loss']:.4f}")
    print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print(f"  Time: {train_metrics['time']:.2f}s")
    
    # Generate config string for filenames
    config_str = f"ca{int(USE_CLASS_AWARE_SAMPLING)}_mo{int(USE_MOSAIC_AUGMENTATION)}_rn{int(USE_RANDOM_AUGMENTATION)}"
    
    # Save best model
    if val_metrics['val_loss'] < best_val_loss:
        best_val_loss = val_metrics['val_loss']
        checkpoint_path = os.path.join(CHECKPOINT_DIR, f"best_model_{config_str}.pth")
        save_model_checkpoint(
            model, optimizer, epoch + 1, val_metrics['val_loss'],
            checkpoint_path,
            additional_info={
                'train_loss': train_metrics['loss'],
                'config': config_str
            }
        )
        print(f"  ✓ New best model saved!")
    
    # Save latest checkpoint
    latest_path = os.path.join(CHECKPOINT_DIR, f"latest_model_{config_str}.pth")
    save_model_checkpoint(
        model, optimizer, epoch + 1, val_metrics['val_loss'],
        latest_path
    )

print("\n" + "="*60)
print("Training completed!")
print("="*60)

## 10. Plot Training History

In [ ]:
plot_training_history({
    'Training Loss': history['train_loss'],
    'Validation Loss': history['val_loss'],
    'Learning Rate': history['learning_rate']
})

## 11. Final Evaluation

In [ ]:
# Prepare evaluation dataset and loader
eval_dataset = test_dataset if has_test_set else val_dataset
eval_loader = data.DataLoader(
    eval_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)

print(f"Evaluating on: {'Test' if has_test_set else 'Validation'} set")

# Load best model for this config
config_str = f"ca{int(USE_CLASS_AWARE_SAMPLING)}_mo{int(USE_MOSAIC_AUGMENTATION)}_rn{int(USE_RANDOM_AUGMENTATION)}"
best_checkpoint_path = os.path.join(CHECKPOINT_DIR, f"best_model_{config_str}.pth")
best_checkpoint = torch.load(best_checkpoint_path)
model.load_state_dict(best_checkpoint['model_state_dict'])
model.to(DEVICE)

print(f"Loaded best model from epoch {best_checkpoint['epoch']}")

# Evaluate
detailed_results = evaluate_map(
    model, eval_loader, DEVICE,
    iou_threshold=0.5,
    confidence_threshold=0.05
)

print("\nDetailed Results at IoU=0.5:")
print_metrics(detailed_results, class_names=eval_dataset.categories)

## 12. Save Results

In [ ]:
# Save evaluation results with unique filename
config_str = f"ca{int(USE_CLASS_AWARE_SAMPLING)}_mo{int(USE_MOSAIC_AUGMENTATION)}_rn{int(USE_RANDOM_AUGMENTATION)}"
results_file = f"results_{config_str}.json"

results = {
    'model': 'Faster R-CNN ResNet50-FPN',
    'config': {
        'class_aware_sampling': USE_CLASS_AWARE_SAMPLING,
        'mosaic_augmentation': USE_MOSAIC_AUGMENTATION,
        'random_augmentation': USE_RANDOM_AUGMENTATION,
        'mosaic_prob': MOSAIC_PROB
    },
    'evaluated_on': 'test' if has_test_set else 'validation',
    'num_epochs': NUM_EPOCHS,
    'batch_size': BATCH_SIZE,
    'best_val_loss': float(best_val_loss),
    'final_train_loss': float(history['train_loss'][-1]),
    'mAP@0.5': float(detailed_results['mAP']),
    'per_class_ap': {k: float(v) for k, v in detailed_results.items() if k.startswith('AP_class_')},
    'training_history': history
}

with open(results_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Results saved to {results_file}")
print(f"You can compare different runs by changing the flags at the top and re-running.")